In [1]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModel, get_scheduler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import pandas as pd
from tqdm import tqdm


In [2]:
model_name = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_df = pd.read_csv("/content/trac2_CONVT_train.csv", sep=',', engine='python', on_bad_lines='skip', escapechar='\\')
dev_df = pd.read_csv("/content/trac2_CONVT_dev.csv", sep=',', engine='python', on_bad_lines='skip', escapechar='\\')
test_df = pd.read_csv("/content/trac2_CONVT_test.csv", sep=',', engine='python', on_bad_lines='skip', escapechar='\\')

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_texts(texts):
  return tokenizer(
      list(texts),
      truncation=True,
      padding="max_length",
      max_length=128,
      return_tensors="pt"
  )
train_tokens = tokenize_texts(train_df["text"])
dev_tokens = tokenize_texts(dev_df["text"])
test_tokens = tokenize_texts(test_df["text"])
for col in ["Emotion", "EmotionalPolarity"]:
  train_df[col] = train_df[col].astype(int) - train_df[col].min()
  dev_df[col] = dev_df[col].astype(int) - dev_df[col].min()
train_emotion = torch.tensor(train_df["Emotion"].values, dtype=torch.float)
train_polarity = torch.tensor(train_df["EmotionalPolarity"].values, dtype=torch.long)
train_empathy = torch.tensor(train_df["Empathy"].values, dtype=torch.float)
dev_emotion = torch.tensor(dev_df["Emotion"].values, dtype=torch.float)
dev_emotional_polarity = torch.tensor(dev_df["Emotion"].values, dtype=torch.long)
dev_empathy = torch.tensor(dev_df["Emotion"].values, dtype=torch.float)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
class MultiTaskModel(nn.Module):
  def __init__(self, model_name, polarities):
    super().__init__()
    self.bert = AutoModel.from_pretrained(model_name)
    hidden_size = self.bert.config.hidden_size
    dropout = self.bert.config.hidden_dropout_prob
    self.drop = nn.Dropout(dropout)
    self.emotion_classifier = nn.Linear(hidden_size, 1)
    self.polarity_classifier = nn.Linear(hidden_size, polarities)
    self.empathy = nn.Linear(hidden_size, 1)
  def forward(self, input_id, attention_mask):
    outputs = self.bert(input_ids=input_id, attention_mask=attention_mask)
    cls_output = self.drop(outputs.last_hidden_state[:, 0, :])
    emotion = self.emotion_classifier(cls_output).squeeze(-1)
    polarity = self.polarity_classifier(cls_output)
    empathy = self.empathy(cls_output).squeeze(-1)
    return emotion, polarity, empathy


In [5]:
train_dataset = TensorDataset(
    train_tokens["input_ids"], train_tokens["attention_mask"], train_emotion, train_polarity, train_empathy
)
dev_dataset = TensorDataset(
    dev_tokens["input_ids"], dev_tokens["attention_mask"], dev_emotion, dev_emotional_polarity, dev_empathy
)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16)

In [7]:
num_polar_labels = len(sorted(train_df["EmotionalPolarity"].unique()))
model = MultiTaskModel(model_name, num_polar_labels).to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [8]:
def train_epochs(model, dataloader, optimizer, scheduler):
  model.train()
  total_loss = 0.0
  emotion_loss = nn.MSELoss()
  polarity_loss = nn.CrossEntropyLoss()
  empathy_loss = nn.MSELoss()
  for batch in tqdm(dataloader, desc="Training", leave=False):
    input_ids, attention_mask, emo_labl, emo_pol_lab, emp_lbl = [x.to(device) for x in batch]
    optimizer.zero_grad()
    emo_logits, emo_pol_logits, emp_logits = model(input_ids, attention_mask)
    loss = emotion_loss(emo_logits, emo_labl) + polarity_loss(emo_pol_logits, emo_pol_lab) + empathy_loss(emp_logits, emp_lbl)
    loss.backward()
    optimizer.step()
    scheduler.step()
    total_loss += loss.item()
  return total_loss / len(dataloader)


In [9]:
def evaluate(model, dataloader):
  model.eval()
  emo_preds, emo_pol_preds, emp_preds = [], [], []
  emo_lbls, emo_pol_lbls, emp_lbls = [], [], []
  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluating", leave=False):
      input_ids, attention_mask, emo_labl, emo_pol_lab, emp_lbl = [x.to(device) for x in batch]
      emo_logits, emo_pol_logits, emp_logits = model(input_ids, attention_mask)

      emo_preds.extend(emo_logits.cpu().numpy().tolist())
      emo_pol_preds.extend(torch.argmax(emo_pol_logits, dim=1).cpu().numpy().tolist())
      emp_preds.extend(emp_logits.cpu().numpy().tolist())

      emo_lbls.extend(emo_labl.cpu().numpy().tolist())
      emo_pol_lbls.extend(emo_pol_lab.cpu().numpy().tolist())
      emp_lbls.extend(emp_lbl.cpu().numpy().tolist())
  return {
      "Emotion_MAE": mean_absolute_error(emo_lbls, emo_preds),
      "Emotional_Polarity_Accuracy": accuracy_score(emo_pol_lbls, emo_pol_preds),
      "Empathy_MAE": mean_absolute_error(emp_lbls, emp_preds),
      "Polarity_F1": f1_score(emo_pol_lbls, emo_pol_preds, average="weighted")

  }

In [11]:
for epoch in range(num_epochs):
  train_loss = train_epochs(model, train_loader, optimizer, lr_scheduler)
  metrics = evaluate(model, dev_loader)
  print(f"\nEpoch {epoch+1}/{num_epochs}")
  print(f"Train Loss: {train_loss:.4f}")
  print(f"Emotion MAE: {metrics['Emotion_MAE']:.4f}")
  print(f"Empathy MAE: {metrics['Empathy_MAE']:.4f}")
  print(f"Emotional Polarity Accuracy: {metrics['Emotional_Polarity_Accuracy']:.4f}, F1: {metrics['Polarity_F1']:.4f}")
  print("-" * 50)



Epoch 1/5
Train Loss: 1.9602
Emotion MAE: 0.4976
Empathy MAE: 0.5512
Emotional Polarity Accuracy: 0.4071, F1: 0.3526
--------------------------------------------------



Epoch 2/5
Train Loss: 1.4755
Emotion MAE: 0.5016
Empathy MAE: 0.5726
Emotional Polarity Accuracy: 0.3859, F1: 0.3369
--------------------------------------------------



Epoch 3/5
Train Loss: 1.3187
Emotion MAE: 0.4844
Empathy MAE: 0.5663
Emotional Polarity Accuracy: 0.3990, F1: 0.3521
--------------------------------------------------



Epoch 4/5
Train Loss: 1.2122
Emotion MAE: 0.5015
Empathy MAE: 0.5937
Emotional Polarity Accuracy: 0.3960, F1: 0.3515
--------------------------------------------------



Epoch 5/5
Train Loss: 1.1368
Emotion MAE: 0.5020
Empathy MAE: 0.6054
Emotional Polarity Accuracy: 0.3808, F1: 0.3358
--------------------------------------------------


In [12]:
model.eval()
emo_preds, emo_pol_preds, emp_preds = [], [], []
test_ids = test_df["id"].tolist()
input_ids, mask = test_tokens["input_ids"], test_tokens["attention_mask"]
with torch.no_grad():
  for i in range(0, len(input_ids), 16):
    batch_ids = input_ids[i:i+16].to(device)
    batch_mask = mask[i:i+16].to(device)
    emo_logits, emo_pol_logits, emp_logits = model(batch_ids, batch_mask)
    emo_preds.extend(emo_logits.cpu().numpy().tolist())
    emo_pol_preds.extend(torch.argmax(emo_pol_logits, dim=1).cpu().numpy().tolist())
    emp_preds.extend(emp_logits.cpu().numpy().tolist())
pred_df = pd.DataFrame({
    "id": test_ids,
    "Emotion": emo_preds,
    "EmotionalPolarity": emo_pol_preds,
    "Empathy": emp_preds
})
pred_df.to_csv("predictions_bert.csv", index=False)
